# mem0ai
mem0是一个开源的记忆库，旨在帮助用户存储和检索信息。\
Mem0 的准确率比 OpenAI Memory 高 26%， 延迟降低 91%， 并节省 90% 的tokens \
其支持 向量数据库、图数据库、自定义LLM、自定义池化模型，并且一次可以存储多种类型的数据。\
地址如下：https://github.com/mem0ai/mem0

## 本地运行
首先启动基本服务：\
1. 使用qdrant作为向量数据库，需要先启动qdrant服务，命令如下：
```bash
docker run -p 6333:6333 -p 6334:6334 qdrant/qdrant:v1.1.2
```
2. 使用 memgraph 作为知识图谱数据库，需要先启动memgraph服务，命令如下：
```bash
docker run -p 7687:7687 memgraph/memgraph-mage:latest --schema-info-enabled=True
Alternatively, you can download Memgraph Lab and connect it to your existing Memgraph instance.
```
3. 使用ollama作为大模型，需要先启动ollama服务，命令如下：
```bash
ollama run qwen3:8b
```
4. 安装依赖pip install "mem0ai[graph]" ,chromadb, langchain_memgraph=0.1.1(2025.5.27 mem0 has bug in this,so install that)



In [7]:
import os
from mem0 import Memory
import os
os.environ["OPENAI_API_KEY"] = "sk-zk2e84a4dda032cfd1817544f88788ee5f966f213b4dbe83"
config = {
  "graph_store": {
        "provider": "memgraph",
        "config": {
            "url": "bolt://localhost:7687",
            "username": "memgraph",
            "password": "xxx",
        },
    },
   "llm": {
        "provider": "ollama",
        "config": {
            "model": "qwen3:8b",
            "temperature": 0,
            "max_tokens": 2000,
            "ollama_base_url": "http://localhost:11434",  # Ensure this URL is correct
        },
    },
     "vector_store": {
        "provider": "chroma",
        "config": {
            "collection_name": "test",
            "path": "db",
        }
    },
     "embedder": {
        "provider": "ollama",
        "config": {
            "model": "nomic-embed-text:latest",
            # Alternatively, you can use "snowflake-arctic-embed:latest"
            "ollama_base_url": "http://localhost:11434",
            "embedding_dims":768
        },
    },
     }

# Initialize Memory with the configuration
m = Memory.from_config(config_dict = config)

# Add a memory
# m.add("I like pizza", user_id="alice")
# m.add("I have travel to China,it is a very beautiful city,and Beijing is its capital ", user_id="alice")
# m.add("I have travel to China,it is a very beautiful city,and Beijing is its capital ", user_id="alice")

# m.add("I Like piano ,and I have ever join the club in university", user_id="Frank")


## 自动创建了一个图谱关系

![image.png](memory.png)

In [11]:
# m.get_all(user_id="alice") # wrong

In [12]:
m.search("tell me my name.", user_id="alice") # search memory of user
m.delete_all(user_id="alice") # delete user memory

{'message': 'Memories deleted successfully!'}